In [27]:
import numpy as np
from matplotlib import pyplot as plt
import netCDF4 as nc
import nibabel as nib
from scipy import special, optimize
import ipywidgets as widgets

# Loading the ensemble

In [2]:
ds_2 = nc.Dataset('../../Data/DEMETER/original format/day20_2meter_temp_C.nc')
ds_200 = nc.Dataset('../../Data/DEMETER/original format/day20_200_temp.nc')
ds_500 = nc.Dataset('../../Data/DEMETER/original format/day20_500_temp.nc')
ds_850 = nc.Dataset('../../Data/DEMETER/original format/day20_850_temp.nc')
heights = [ds_2, ds_200, ds_500, ds_850]
print(ds_2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    history: 2009-05-19 17:02:00 GMT by mars2netcdf-0.92
    dimensions(sizes): longitude(144), latitude(73), number(9), time(1)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 number(number), int32 time(time), int16 cnrm(time, number, latitude, longitude), int16 crfc(time, number, latitude, longitude), int16 lody(time, number, latitude, longitude), int16 scnr(time, number, latitude, longitude), int16 scwf(time, number, latitude, longitude), int16 smpi(time, number, latitude, longitude), int16 ukmo(time, number, latitude, longitude)
    groups: 


In [3]:
simulations = ['cnrm', 'lody']

models = np.empty(( len(simulations), ds_2.dimensions['longitude'].size, ds_2.dimensions['latitude'].size, 4, 9))
for height,ds in enumerate(heights):
    for idx,sim in enumerate(simulations):
        ensemble = ds[sim][0]
        models[idx,:,:,height,:] = ensemble.T

print(models.shape)

(2, 144, 73, 4, 9)


# Standard GV-fuzzy metric

\begin{align*}
    M_d(x, y, t) &= \frac{ t }{ t + d(x,y) }
\end{align*}
while $d(x,y)$ is the Wasserstein metric:
\begin{align*}
    W_p(P, Q) &= \left( \frac{1}{n} \sum\limits^n_{i=1} \Vert X_i - Y_i \Vert^p \right)^{1/p}
\end{align*}

In [24]:
def wasserstein(X, Y, p=2):
    return ( 1 / X.shape[3] * np.sum((X - Y)**p, axis=3) )**(1/p)

def standard_gv_fuzzy(t ,wasserstein_value):
    return np.array([ u / ( u + wasserstein_value ) if u > 0 else u for u in t ])

In [25]:
difference_ensemble = wasserstein(models[0,:,:,:,:], models[1,:,:,:,:])
print(difference_ensemble.shape)

(144, 73, 4)


# Fitting the Gamma CDF

In [26]:
def gamma_cdf(t, k, theta):
    return special.gammainc(k, t / theta) / special.gamma(k)

In [29]:
t = np.linspace(0, 50, 100)


k = np.zeros_like(difference_ensemble)
theta = np.zeros_like(difference_ensemble)
for u in range(difference_ensemble.shape[0]):
    for v in range(difference_ensemble.shape[1]):
        for w in range(difference_ensemble.shape[2]):
            difference = standard_gv_fuzzy(t, difference_ensemble[u, v, w])
            popt, pcov = optimize.curve_fit(gamma_cdf, t, difference)
            k[u, v, w] = popt[0]
            theta[u, v, w] = popt[1]

In [31]:
print('k:', k.shape)
print(r'$\theta$:', theta.shape)

k: (144, 73, 4)
$\theta$: (144, 73, 4)


In [32]:
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
plt.title('k')
plt.imshow(k[:,:,0].T)
plt.colorbar()

plt.subplot(1,2,2)
plt.title(r'$\theta$')
plt.imshow(theta[:,:,0].T)
plt.colorbar()
plt.show()

In [33]:
def gamma_pdf(x, k, theta):
    g = ( x**(k-1) * np.exp( -x / theta ) ) / ( theta**k * special.gamma(k) )
    return g

In [36]:
@widgets.interact(x=(0, k.shape[0]-1, 1), y=(0, k.shape[1]-1, 1))
def show_gamma(x=0, y=0):
    plt.figure(figsize=(18,10))
    plt.subplot(2,1,1)
    plt.imshow(k[:,:,0].T)
    plt.scatter(x, y, c='magenta', s=2)
    plt.colorbar()

    u = np.linspace(1e-9,120)
    plt.subplot(2,1,2)
    plt.plot(u, gamma_pdf(u, k[x, y, 0], theta[x, y, 0]) )
    plt.grid()

    plt.show()

interactive(children=(IntSlider(value=0, description='x', max=143), IntSlider(value=0, description='y', max=72…

In [38]:
affine = np.eye(4)
affine[2,2] = 10
k_image = nib.Nifti1Image(k, affine)
nib.save(k_image, '../../Data/DEMETER/comparison/standard fuzzy distance/k.nii')
np.save('../../Data/DEMETER/comparison/standard fuzzy distance/k.npy', k)

theta_image = nib.Nifti1Image(theta, affine)
nib.save(theta_image, '../../Data/DEMETER/comparison/standard fuzzy distance/theta.nii')
np.save('../../Data/DEMETER/comparison/standard fuzzy distance/theta.npy', theta)